In [4]:
import pandas as pd
import numpy as np

!git clone https://github.com/Elkkujou/Gradu.git
%cd Gradu
!ls


Cloning into 'Gradu'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 2.74 MiB | 7.91 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Gradu
'Financial turbulence.ipynb'   FT_source.xlsx   RF_regime.ipynb   THE_ONE.xlsx


In [12]:

input_file = r"FT_source.xlsx"
sheet_name = "Prices"
output_file = r"C:\\Users\\Elmeri\\OneDrive - Aalto University\\Gradu oscun kaa\\DATA\\Hinta ja return data\\ft_outputs.xlsx"

# Read the Excel sheet
price_data = pd.read_excel(input_file, sheet_name=sheet_name, header=0)  # Read the first row as headers

# Drop the second row (codes)
price_data = price_data.drop(index=1)

In [13]:
price_data

,Unnamed: 0,Information Technology,Energy,Financials,Health Care,Consumer Staples,Consumer Discretionary,Utilities,Industrials,Materials,Communication Services,US 2 Year Treasury Yield,US 10 Year Treasury Yield
0,.TRGSPT,.TRGSPT,.TRGSPE,.TRGSPF,.TRGSPL,.TRGSPS,.TRGSPA,.TRGSPU,.TRGSPD,.TRGSPM,.TRGSPC,DGS2,DGS10
2,1990-01-31 00:00:00,59.8203,71.3038,57.6987,70.4176,49.8539,59.2098,72.2766,59.8329,62.2671,353.9369,8.37,8.47
3,1990-02-28 00:00:00,62.2973,73.0873,59.6877,70.1181,49.1484,57.7162,72.0916,61.2038,62.9716,358.4958,8.63,8.59
4,1990-03-31 00:00:00,64.6819,73.3427,58.038,73.9396,52.1171,59.7416,71.2447,63.0806,64.1002,367.9953,8.72,8.79
5,1990-04-30 00:00:00,64.0974,70.5901,55.6856,71.6275,52.7921,60.0541,67.4247,60.8344,60.9757,358.8147,8.64,8.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,2024-09-30 00:00:00,5707.8946,1538.4981,1418.5681,834.9467,1865.7296,2975.2635,1307.0546,2344.2847,1161.398,12608.0737,3.97,4.1
419,2024-10-31 00:00:00,5652.5334,1550.621,1456.6752,851.1408,1813.5312,2837.6886,1293.6896,2307.9841,1120.9023,12493.7364,4.26,4.36
420,2024-11-30 00:00:00,5915.5226,1658.0848,1606.4475,877.5575,1899.4425,2845.7033,1341.4494,2615.859,1139.192,13227.133,4.23,4.39
421,2024-12-31 00:00:00,5983.9328,1501.0154,1519.1604,908.9956,1804.9655,2668.8758,1234.9902,2678.4201,1017.1049,12911.8218,na,na


In [14]:
# Replace commas with dots and convert numeric columns
price_data = price_data.replace({",": "."}, regex=True)
for col in price_data.columns[1:]:  # Skip the Timestamp column
    price_data[col] = pd.to_numeric(price_data[col], errors='coerce')

# Calculate returns for equity indexes (price changes)
def calculate_returns(df):
    return df.pct_change()  # Calculate percentage change (returns)

equity_columns = price_data.columns[1:-2]  # All columns except Timestamp and the last two (bond yields)
bond_yield_columns = price_data.columns[-2:]  # Last two columns are bond yields
returns_data = calculate_returns(price_data[equity_columns])

# Convert bond yields to returns (difference between consecutive yields)
def convert_yields_to_returns(df):
    return df.diff()

bond_returns = convert_yields_to_returns(price_data[bond_yield_columns])



In [15]:
# Combine equity returns and bond returns
combined_data = pd.concat([returns_data, bond_returns], axis=1)
combined_data.insert(0, "Timestamp", price_data.iloc[1:, 0])  # Add the timestamp column back

# Format the date column
def format_date_column(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name]).dt.strftime('%d.%m.%Y')

format_date_column(combined_data, "Timestamp")

# Drop rows with NaN values (required for covariance matrix calculation)
returns_clean = combined_data.dropna()

# Save renamed returns to the first sheet
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    combined_data.to_excel(writer, index=False, sheet_name="Returns")


combined_data.loc[returns_clean.index, "Turbulence"] = calculate_turbulence(returns_only)

In [16]:
# Calculate financial turbulence
def calculate_turbulence(data):
    results = []
    mu = data.mean()
    covariance_matrix = data.cov()
    try:
        covariance_matrix_inv = np.linalg.inv(covariance_matrix)
    except np.linalg.LinAlgError:
        covariance_matrix_inv = np.linalg.pinv(covariance_matrix)  # Use pseudo-inverse if singular

    for index, row in data.iterrows():
        yt = row - mu
        turbulence = float(yt.T @ covariance_matrix_inv @ yt)
        results.append(turbulence)
    return results

# Subset the returns columns (excluding Timestamp)
returns_only = returns_clean.iloc[:, 1:]

# Calculate turbulence
combined_data["Turbulence"] = np.nan  # Initialize turbulence column

In [17]:
# Save intermediate outputs: correlation matrix, mean, covariance matrix
correlation_matrix = returns_only.corr()
mu = returns_only.mean()
covariance_matrix = returns_only.cov()

with pd.ExcelWriter(output_file, engine="openpyxl", mode="a") as writer:
    correlation_matrix.to_excel(writer, index=True, sheet_name="Correlations")
    mu.to_excel(writer, index=True, header=["Mean"], sheet_name="Mean")
    covariance_matrix.to_excel(writer, index=True, sheet_name="Covariance Matrix")

# Save turbulence data to the final sheet
turbulence_output = combined_data[["Timestamp", "Turbulence"]]
with pd.ExcelWriter(output_file, engine="openpyxl", mode="a") as writer:
    turbulence_output.to_excel(writer, index=False, sheet_name="Turbulence")

print(f"Financial turbulence and intermediate steps saved to {output_file}.")

Financial turbulence and intermediate steps saved to C:\\Users\\Elmeri\\OneDrive - Aalto University\\Gradu oscun kaa\\DATA\\Hinta ja return data\\ft_outputs.xlsx.
